In this notebook, I perform Ramsey fringe experiment for the (01) and (12) subspace.

### References

[1]: M. J. Peterer *et al.* Coherence and Decay of Higher Energy Levels of a Superconducting Transmon Qubit. *Phys. Rev. Lett.* **114***, 010501 (2015). [https://doi.org/10.1103/PhysRevLett.114.010501](https://doi.org/10.1103/PhysRevLett.114.010501).

## Dependencies

In [1]:
%config InlineBackend.figure_formats = ['svg']

import os
os.chdir('/Users/ngdnhtien/Codespace/PulsatingPulseShop/')

# qiskit
from qiskit import pulse
from qiskit.circuit import Parameter
from qiskit.circuit import QuantumCircuit, Gate
from qiskit_ibm_provider import IBMProvider

# qutritium
from utility import *
from constant import *

# numerical
import numpy as np
from scipy.optimize import curve_fit
import pickle 

# plot
from matplotlib.ticker import MultipleLocator

plt.rcParams['axes.linewidth'] = 1.25

In [2]:
provider = IBMProvider()
print(provider.backends())

backend = provider.get_backend('ibm_brisbane')

backend_config = backend.configuration()
backend_defaults = backend.defaults()
backend_properties = backend.properties()
dt = backend_config.dt

qubit = 0
cbit = 0
num_qubits = 1
num_cbits = 1
weight = 1

qubit_01_freq = backend_defaults.qubit_freq_est[qubit]
qubit_anharmonicty = backend_properties.qubits[qubit][3].value*GHz
qubit_12_freq = qubit_01_freq + qubit_anharmonicty

[<IBMBackend('ibmq_qasm_simulator')>, <IBMBackend('simulator_extended_stabilizer')>, <IBMBackend('simulator_mps')>, <IBMBackend('simulator_statevector')>, <IBMBackend('simulator_stabilizer')>, <IBMBackend('ibm_brisbane')>, <IBMBackend('ibm_kyoto')>, <IBMBackend('ibm_osaka')>]


In [4]:
p12_amp = 0.24029996372238208 # 3rd Dec 23
# 0.23743549524170637

with pulse.build(backend=backend) as hp12_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(qubit_12_freq, drive_chan)
    pulse.play(pulse.Drag(duration=160, amp=p12_amp/2, sigma=40, beta=2.0, name='hp12'), drive_chan)
    
hp12_gate = Gate(r'X_{\pi/2}^{(12)}', weight, [])
    
with pulse.build(backend=backend) as p12_sched:
    drive_chan = qiskit.pulse.drive_channel(qubit)
    pulse.set_frequency(qubit_12_freq, drive_chan)
    pulse.play(pulse.Drag(duration=160, amp=p12_amp, sigma=40, beta=2.0, name='p12'), drive_chan)

p12_gate = Gate(r'X_{\pi}^{(12)}', weight, [])

In [8]:
with pulse.build(backend=backend, default_alignment='left') as meas_pulse:
    meas_chan = pulse.measure_channel(qubit)
    pulse.shift_frequency(-1.0*MHz, meas_chan)
    ac_chan = pulse.acquire_channel(qubit)
    mem_chan = pulse.MemorySlot(qubit)
    pulse.acquire(1440, ac_chan, mem_chan)
    pulse.play(pulse.GaussianSquare(amp=0.6,
                                    duration=1440,
                                    sigma=32,
                                    width=1312,
                                    angle=0.7650583861061049), meas_chan)
    pulse.delay(6560, meas_chan)

## Ramsey pseudo-interferometer experiment 